In [21]:
import pinocchio as pin

 
urdf_filename = "Exo/description.urdf"
 
model    = pin.buildModelFromUrdf(urdf_filename)

print('model name: ' + model.name)
 
# Create data required by the algorithms
data     = model.createData()
 
# Sample a random configuration
q        = pin.randomConfiguration(model)
print('q: %s' % q.T)
 
# Perform the forward kinematics over the kinematic tree
pin.forwardKinematics(model,data,q)



# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}"
          .format( name, *oMi.translation.T.flat )))

model name: EXO_TAO
q: [-0.53901383  0.6294827 ]
universe                 :  0.00  0.00  0.00
r_bar                    :  0.00  0.00  0.84
r_knee                   :  0.24 -0.22  0.48


In [31]:
M = pin.crba(model,data,q)

print(M)

[[0. 0.]
 [0. 0.]]
